In [3]:
import altair as alt
from google.colab import auth
auth.authenticate_user()

In [Part 1](https://colab.research.google.com/github/rviscomi/har.fyi/blob/main/src/assets/exploring_httparchive-all-pages_tables.ipynb) we examined the `pages` tables. Then in [Part 2](https://colab.research.google.com/github/rviscomi/har.fyi/blob/main/src/assets/exploring_httparchive-all-requests_tables.ipynb) we worked with the `requests` tables. Now let's look at how we can combine both the `requests` and `pages` tables to perform a deeper analysis. In the following example, we're going to investigate the following question:
  "Is image weight more commonly a factor of 1 large image, or many smaller images?".

Let's start with a simple query against the `pages` table. This is a simple query that just selects the URL, the number of images and the bytes of those images.

In [4]:
# This query will process 25 GB when run.
%%bigquery --project httparchive
SELECT
  page,
  CAST(JSON_VALUE(summary, '$.reqImg') AS INT64) AS image_requests,
  CAST(JSON_VALUE(summary, '$.bytesImg') AS INT64) AS image_bytes
FROM `httparchive.all.pages`
WHERE date = '2024-05-01'
    AND client='desktop'
    AND is_root_page
ORDER BY
  image_bytes DESC

Query is running:   0%|          |

Downloading:   0%|          |

,page,image_requests,image_bytes
0,https://www.laurel.edu/,170,143775693
1,https://www.salsayotra.fr/,223,120086614
2,https://www.traumama.com/,65,116583274
3,https://gridarchitects.co.uk/,349,110498520
4,https://studfist.fans/,230,107665948
...,...,...,...
12900235,http://www.unam.me/,<NA>,<NA>
12900236,http://simpro.jayabeton.com/,<NA>,<NA>
12900237,http://moncrane.mn/,<NA>,<NA>
12900238,http://iptu.sorocaba.sp.gov.br/,<NA>,<NA>


In the previous example we didn't use the requests table at all. However if we wanted to, we could do the same analysis by JOINing the request table. In order to do this, I've made the following modifications to the query above:

* Named the tables (i.e., so we can reference "httparchive.all.pages" as "pages")
* Reference the tables for some of the columns that exist in both tables (such as page)
* Since we're focusing on images, we'll include `requests.type="image"` in our WHERE clause.
* JOIN the tables `ON requests.page = pages.page`

The below query is similar to the previous one, but uses both tables. The unique differentiation here is that `requests` table is much bigger and we are applying sampling to the `requests` table for a faster and cheaper result.

In [4]:
# This query will process 80 GB when run.
%%bigquery --project httparchive
WITH pages AS (
  SELECT
    page
  FROM `httparchive.all.pages`
  WHERE date = '2024-05-01'
      AND client='desktop'
      AND is_root_page
), requests AS (
  SELECT
    page,
    COUNT(0) AS image_requests,
    SUM(CAST(JSON_VALUE(summary, '$.respBodySize') AS INT64)) AS image_bytes
  FROM `httparchive.all.requests` TABLESAMPLE SYSTEM (10 PERCENT)
  WHERE date = '2024-05-01'
      AND client='desktop'
      AND is_root_page
      AND type = "image"
  GROUP BY
    page
)

SELECT
  pages.page,
  image_requests,
  image_bytes
FROM requests
LEFT JOIN pages
ON requests.page = pages.page
ORDER BY
  image_bytes DESC

Query is running:   0%|          |

Downloading:   0%|          |

,page,image_requests,image_bytes
0,https://princetel.com/,1,69269304
1,https://hopeshasta.org/,1,68550656
2,https://innoviz.tech/,2,68533708
3,https://www.sassandbelle.co.uk/,6,68387227
4,https://www.radiocolor.ro/,3,68317514
...,...,...,...
10102030,https://niagarasushiandramen.com/,1,<NA>
10102031,https://obituarios.elsalvador.com/,1,<NA>
10102032,https://www.qcwxjs.com/,1,<NA>
10102033,https://carispesifikasi.com/,1,<NA>


Some of these image weights seem quite extreme, but these represent the most image heavy sites in the archive. If you want to validate this, you can load one of the pages in DevTools or WebPageTest and see for yourself.  For example based on the results above, https://princetel.com/ has 1 image totaling 110MB. WebPageTest confirms that this is accurate (although the image weight likely changed slightly between when the HTTP Archive saw the page vs when this manual test was performed.

# Content breakdown by mime/type

In [6]:
# This query will process 131 GB when run.
%%bigquery df_requests_type --project httparchive
SELECT
    type,
    COUNT(0) AS requests,
    SUM(CAST(JSON_VALUE(summary, '$.respBodySize') AS INT64)) AS bytes
FROM `httparchive.all.requests` TABLESAMPLE SYSTEM (5 PERCENT)
WHERE date = '2024-05-01'
    AND client='desktop'
    AND is_root_page
    AND page='https://princetel.com/'
GROUP BY
    type
ORDER BY
    requests DESC

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
df_requests_type.head()

,type,requests,bytes
0,font,1,7884
1,script,1,5004
2,image,1,47302


In [48]:
pie_chart_requests = alt.Chart(df_requests_type).mark_arc().encode(
    theta=alt.Theta(field="requests", type="quantitative", stack=True),
    color=alt.Color(field="type", type="nominal", sort=None),
    order=alt.Order(field="requests", type="quantitative", sort='descending'),
    tooltip=["type", alt.Tooltip("requests:Q", format=".0")]
).properties(
    title="Requests by Type"
)
pie_chart_requests + pie_chart_requests.mark_text(
    radius=160
).encode(
    text=alt.Text("requests:Q", format=".0")
)

alt.LayerChart(...)

In [47]:
pie_chart_bytes = alt.Chart(df_requests_type).mark_arc().encode(
    theta=alt.Theta(field="bytes", type="quantitative", stack=True),
    color=alt.Color(field="type", type="nominal", sort=None),
    order=alt.Order(field="bytes", type="quantitative", sort='descending'),
    tooltip=["type", alt.Tooltip("bytes:Q", format=".1s")]
).properties(
    title="Bytes by Type"
)
pie_chart_bytes + pie_chart_bytes.mark_text(
    radius=160
).encode(
    text=alt.Text("bytes:Q", format=".1s")
)

alt.LayerChart(...)

If this was all the information we needed, then JOINING the `pages` table would have been unecessary. All the requests data can be aggregated using `page` column in `requests` table.

Another example we will look into is the requests for JS scripts and page `_TotalBlockingTime` metric within the `payload` column. We are using the `page` column to classify whether the `requests` data is first or third party content.

The query below gets the median Total Blocking Time (TBT) per a number of first and third-party scripts.

In [175]:
# This query will process 47 GB when run.
%%bigquery df_tbt_scripts --project httparchive
WITH pages AS (
  SELECT
    page,
    CAST(ROUND(CAST(JSON_VALUE(payload, '$._TotalBlockingTime') AS FLOAT64)/100)*100 AS INT64) AS TBT_bucket
  FROM `httparchive.all.pages` TABLESAMPLE SYSTEM (1 PERCENT)
  WHERE date = '2024-05-01'
      AND client='desktop'
      AND is_root_page
  GROUP BY
    page,
    TBT_bucket
), requests AS (
  SELECT
    page,
    NET.REG_DOMAIN(url) = NET.REG_DOMAIN(page) AS is_1p_script,
    COUNT(0) AS count
  FROM `httparchive.all.requests` TABLESAMPLE SYSTEM (10 PERCENT)
  WHERE date = '2024-05-01'
      AND client='desktop'
      AND is_root_page
      AND type = "script"
  GROUP BY
    page,
    is_1p_script
  HAVING is_1p_script IS NOT NULL
)

SELECT
  is_1p_script,
  count,
  APPROX_QUANTILES(TBT_bucket, 100)[50] AS TBT_median
FROM requests
INNER JOIN pages
ON requests.page = pages.page
GROUP BY
  count,
  is_1p_script
ORDER BY
  count,
  is_1p_script

Query is running:   0%|          |

Downloading:   0%|          |

In [176]:
df_tbt_scripts

,is_1p_script,count,TBT_median
0,False,1,100
1,True,1,100
2,False,2,100
3,True,2,100
4,False,3,200
...,...,...,...
122,True,100,300
123,True,134,1100
124,False,226,0
125,False,255,0


In [177]:
# filter outliers
df_filtered = df_tbt_scripts[(df_tbt_scripts['TBT_median'] < 20000) & (df_tbt_scripts['count'] < 60)]

base = alt.Chart(df_filtered).mark_circle(opacity=0.5).encode(
    alt.X('count:Q').title('Script Count'),
    alt.Y('TBT_median:Q').axis(format='s').title('Total Blocking Time, ms'),
    alt.Color('is_1p_script:N').title('1st Party Script'),
    tooltip=['TBT_median', 'is_1p_script', 'count']
).properties(
    title="Total Blocking Time (ms) by 1st/3rd Script Count",
    width=1200,
    height=600
)

base + base.transform_loess('count', 'TBT_median', groupby=['is_1p_script']).mark_line()

alt.LayerChart(...)

**Note**: as the query in this guide is sampled the results may misrepresent the true metrics values.

General trend is that 3rd-party scripts tend to have a bigger correllation with the TBT than 1st-party. The TBT tends to increase as the number of 3rd-party scripts increases up to 20 scripts of a particular type per page.

After 20 scripts per page the TBT increases more 3rd-party scripts added to a page.

You can also make a copy of the workbook and experiment with some of your own visualization ideas for the data as well.